## tensorflow转torch
- 安装python3.7     
- 安装tensorflow1.13.1      
- 运行下面代码      
这里先运行tensorflow的环境

In [9]:
import h5py
with h5py.File(r'E:\Pystack\PyStack\data\Models\turn\weights.leaf_nodes.hdf5', 'r') as f:
    print(list(f.keys()))

OSError: Unable to open file (file signature not found)

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
import sys 

sys.path.append(r'E:\Pystack\PyStack\src')

def BasicHuberLoss(delta=1.0):
	def loss(y_true, y_pred):
		return tf.compat.v1.losses.huber_loss(y_true, y_pred, delta=delta)
	return loss

def masked_huber_loss(y_true, y_pred):
	loss = tf.compat.v1.losses.huber_loss(y_true, y_pred, delta=1.0)
	zero = tf.constant(0.0, dtype=tf.float32)
	batch_size = tf.cast(tf.shape(y_true)[0], dtype=tf.float32)
	feature_size = tf.cast(tf.shape(y_true)[1], dtype=tf.float32)
	mask = tf.where(tf.equal(y_true, zero), tf.zeros_like(y_true), tf.ones_like(y_true))
	loss_multiplier = (batch_size * feature_size) / tf.reduce_sum(mask)
	return loss * loss_multiplier

# ✅ 加入 tf 到 custom_objects 里
custom_objects = {
	'tf': tf,
	'loss': BasicHuberLoss(delta=1.0),
	'masked_huber_loss': masked_huber_loss
}

model_path = r'E:\Pystack\PyStack\data\Models\turn\weights.leaf_nodes.hdf5'
model = load_model(model_path, custom_objects=custom_objects, compile=False)

# 提取模型中的每层权重
npz_dict = {}
for i, layer in enumerate(model.layers):
    w = layer.get_weights()
    if w:
        for idx, weight in enumerate(w):
            npz_dict[f"{layer.name}_{idx}"] = weight
        print(f"Saved weights for layer: {layer.name}")

np.savez('keras_weights_leaf_nodes.npz', **npz_dict)
print("Weights successfully saved to keras_weights_leaf_nodes.npz")


OSError: Unable to open file (file signature not found)

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 2722)         0                                            
__________________________________________________________________________________________________
dense_0 (Dense)                 (None, 500)          1361500     input[0][0]                      
__________________________________________________________________________________________________
dropout_0 (Dropout)             (None, 500)          0           dense_0[0][0]                    
__________________________________________________________________________________________________
batch_norm_0 (BatchNormalizatio (None, 500)          2000        dropout_0[0][0]                  
__________________________________________________________________________________________________
relu_0 (PR

In [2]:
import numpy as np 
weights = np.load(r"E:\Pystack\PyStack\data\Models\river\keras_weights_root_nodes.npz")
print(weights.files)
# print(weights['feed_forward_output_0'].shape)  # 原始权重 shape
# print(weights['feed_forward_output_1'].shape)  # 偏置 shape

['dense_0_0', 'dense_0_1', 'batch_norm_0_0', 'batch_norm_0_1', 'batch_norm_0_2', 'batch_norm_0_3', 'relu_0_0', 'dense_1_0', 'dense_1_1', 'batch_norm_1_0', 'batch_norm_1_1', 'batch_norm_1_2', 'batch_norm_1_3', 'relu_1_0', 'dense_2_0', 'dense_2_1', 'batch_norm_2_0', 'batch_norm_2_1', 'batch_norm_2_2', 'batch_norm_2_3', 'relu_2_0', 'dense_3_0', 'dense_3_1', 'batch_norm_3_0', 'batch_norm_3_1', 'batch_norm_3_2', 'batch_norm_3_3', 'relu_3_0', 'feed_forward_output_0', 'feed_forward_output_1']


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ValueNN(nn.Module):
    def __init__(self, input_size, output_size, num_neurons_list, dropout_rate=0.1):
        super(ValueNN, self).__init__()
        self.output_size = output_size
        self.num_layers = len(num_neurons_list)

        layers = []
        in_features = input_size
        for i, neurons in enumerate(num_neurons_list):
            layers.append(nn.Linear(in_features, neurons))
            layers.append(nn.Dropout(dropout_rate))
            layers.append(nn.BatchNorm1d(neurons))
            layers.append(nn.PReLU(num_parameters=neurons))
            in_features = neurons
        self.ff_layers = nn.Sequential(*layers)

        # 最后一层输出
        self.feed_forward_output = nn.Linear(in_features, output_size)

    def forward(self, x):
        # x: [batch_size, input_size]
        # 取输入前 output_size 维度作为 ranges
        ranges = x[:, :self.output_size]  # [batch, output_size]

        # mask: ranges > 0 => 1 else 0
        mask = (ranges > 0).float()

        # feed forward
        ff_out = self.ff_layers(x)  # [batch, last_layer_size]
        ff_out = self.feed_forward_output(ff_out)  # [batch, output_size]

        # masked_output = ff_out * mask
        masked_output = ff_out * mask

        # estimated_values = dot product along output_size dim: sum(masked_output * ranges, dim=1, keepdim=True)
        estimated_values = torch.sum(masked_output * ranges, dim=1, keepdim=True)

        # division_by_2
        division_by_2 = estimated_values / 2.0

        # zero_sum_output = masked_output - division_by_2 (广播 subtraction)
        zero_sum_output = masked_output - division_by_2

        return zero_sum_output

d:\Anacoda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import numpy as np
import torch

def load_weights_from_keras_npz(model, npz_path):
    weights = np.load(npz_path)

    for i in range(4):  # 4 个隐藏层
        w = torch.tensor(weights[f'dense_{i}_0'].T)
        b = torch.tensor(weights[f'dense_{i}_1'])
        # ff_layers 中每层4个module，线性层是第4*i
        model.ff_layers[4*i].weight.data.copy_(w)
        model.ff_layers[4*i].bias.data.copy_(b)

        gamma = torch.tensor(weights[f'batch_norm_{i}_0'])
        beta = torch.tensor(weights[f'batch_norm_{i}_1'])
        running_mean = torch.tensor(weights[f'batch_norm_{i}_2'])
        running_var = torch.tensor(weights[f'batch_norm_{i}_3'])
        bn_layer = model.ff_layers[4*i + 2]
        bn_layer.weight.data.copy_(gamma)
        bn_layer.bias.data.copy_(beta)
        bn_layer.running_mean.data.copy_(running_mean)
        bn_layer.running_var.data.copy_(running_var)

        prelu_weight = torch.tensor(weights[f'relu_{i}_0'])
        prelu_layer = model.ff_layers[4*i + 3]
        prelu_layer.weight.data.copy_(prelu_weight)

    # 载入最后一层（单独定义）
    w = torch.tensor(weights['feed_forward_output_0'].T)
    b = torch.tensor(weights['feed_forward_output_1'])
    model.feed_forward_output.weight.data.copy_(w)
    model.feed_forward_output.bias.data.copy_(b)

    print("All weights loaded successfully.")


# 用法示例
model = ValueNN(input_size=2722, output_size=2652, num_neurons_list=[500, 500, 500, 500])
load_weights_from_keras_npz(model, r"E:\Pystack\PyStack\data\Models\turn\keras_weights_root_nodes.npz")


All weights loaded successfully.


In [8]:
torch.save(model.state_dict(), 'keras_weights_root_nodes.pth')
